In [4]:
%pip install requests
%pip install PyPDF2
%pip install python-docx

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (145 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.

In [ ]:
import json
import requests
import PyPDF2
import docx
import sys

# Check if two arguments are provided
if len(sys.argv) != 3:
    print("Usage: python script.py <resumepath> <jobprompt>")
    sys.exit(1)

# Get arguments
resumepath = sys.argv[1]
jobprompt = sys.argv[2]

def extract_text_from_file(resumepath: str) -> str:
    if resumepath.endswith(".pdf"):
        with open(resumepath, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif resumepath.endswith(".docx") or resumepath.endswith(".doc"):
        doc = docx.Document(resumepath)
        text = "\n".join([para.text for para in doc.paragraphs])
    else:
        raise ValueError("Unsupported file format")
    return text

def analyze_resume(resumepath: str, jobprompt: str):
    resume_text = extract_text_from_file(resumepath)
    
    ollama_url = "http://127.0.0.1:11434/api/generate"
    model = "deepseek-coder:latest"
    
    prompt = f"""
    Analyze the following resume for the given job prompt and provide a JSON response with the following structure:
    {{
        "rating": <integer from 0 to 10>,
        "exceptional_qualities": [<list of exceptional qualities>],
        "bad_habits": [<list of bad habits detected>]
    }}
    
    Job Prompt:
    {jobprompt}
    
    Resume:
    {resume_text}
    """
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    
    response = requests.post(ollama_url, json=payload)
    
    if response.status_code == 200:
        try:
            data = response.json()
            content = data.get("response", "{}")  # Get response content
            return json.loads(content)  # Parse JSON output
        except json.JSONDecodeError:
            return {"error": "Failed to parse JSON response from Ollama."}
    else:
        return {"error": f"Ollama API error: {response.status_code}"}

if __name__ == "__main__":
    result = analyze_resume(resumepath, jobprompt)
    print(json.dumps(result, indent=4))


Usage: python script.py <arg1> <arg2>


SystemExit: 1

/home/vikk/Documents/GitHub/Innoverse2.0Submission/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
